In [ ]:
pip install highlight_text geoplot cartopy geodatasets plotly nbformat


Note: you may need to restart the kernel to use updated packages.


In [1]:
# data manipulation
import numpy as np
import pandas as pd
import geopandas as gpd
import geodatasets

# visualization
import matplotlib.pyplot as plt
from matplotlib import font_manager
from matplotlib.font_manager import FontProperties
# from highlight_text import fig_text, ax_text
from matplotlib.patches import FancyArrowPatch

import geoplot
import geoplot.crs as gcrs

import plotly.graph_objects as go
import plotly.express as px

import nbformat as nbf
import os

In [9]:
parks_path = 'parks.csv'
parks = pd.read_csv(parks_path)
parks.head()

,주소,운영구분,총주차면,평일유료,토요일유료,공휴일유료,평일시작,평일종료,토요일시작,토요일종료,공휴일시작,공휴일종료,경도,위도,1시간 요금
0,강남구 개포동 126-2,1,132.0,Y,Y,Y,09:00:00,19:00:00,00:00:00,00:00:00,00:00:00,00:00:00,127.066477,37.477263,2400.0
1,강남구 개포동 1266-0,1,97.0,Y,N,N,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,127.048218,37.481496,2400.0
2,강남구 개포동 13-2,1,168.0,Y,N,N,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,127.079307,37.494938,1200.0
3,강남구 개포동 567-23,1,92.0,Y,Y,Y,09:00:00,19:00:00,00:00:00,00:00:00,00:00:00,00:00:00,127.065835,37.477888,2400.0
4,강남구 논현동 168-0,1,192.0,Y,N,N,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,00:00:00,127.026290,37.508171,3600.0


In [ ]:
reports_path = 'reports.csv'
reports = pd.read_csv(reports_path)
reports.head()

,민원접수일,민원접수시간,주소,경도,위도,요일
0,2021-09-29,19:29:00,서울특별시 강서구 강서로15길 49,126.843247,37.532089,Weekday
1,2021-09-29,18:48:00,성북구 오패산로19길 34-5,127.033761,37.609537,Weekday
2,2021-09-29,18:47:00,장위로21다길 59-19 주소지 앞도로 외,127.045741,37.616406,Weekday
3,2021-09-29,18:47:00,서울특별시 강북구 오패산로30길 13,127.034685,37.613820,Weekday
4,2021-09-29,18:46:00,서울특별시 강서구 강서로18길 52-5,126.848703,37.534293,Weekday


In [5]:
print(parks.shape)
print(reports.shape)

(1463, 15)
(3064538, 6)


In [10]:
lat_parks = parks['위도']
lon_parks = parks['경도']
area_parks = parks['총주차면']  # Replace with your actual column name
fee_parks = parks['1시간 요금']
text = '총주차면'+parks['총주차면'].astype(str)  # Optional: for hover info


In [11]:
sizeref = 2. * max(area_parks) / (40**2)  # 40 is desired max marker size in px


In [8]:
text

0       총주차면132.0
1        총주차면97.0
2       총주차면168.0
3        총주차면92.0
4       총주차면192.0
          ...    
1458     총주차면20.0
1459     총주차면56.0
1460    총주차면119.0
1461    총주차면567.0
1462    총주차면107.0
Name: 총주차면, Length: 1463, dtype: object

In [21]:
import plotly.express as px
# parks = px.data.gapminder().query("year == 2007")
locations = parks[['위도', '경도']]
fig = px.scatter_map(parks, lat = '위도', lon = '경도',
                        color="1시간 요금", # which column to use to set the color of markers
                        size="총주차면", size_max=50,
                        zoom=11,
                     )
                     
fig.update_layout(map_style="open-street-map")

fig.show()

---

In [5]:
parks_reports_path = 'parks_complaints.csv'
parks_reports = pd.read_csv(parks_reports_path)
parks_reports.head()


,주소,총주차면,경도,위도,민원 수
0,강남구 개포동 126-2,132.0,127.066477,37.477263,324
1,강남구 개포동 1266-0,97.0,127.048218,37.481496,5464
2,강남구 개포동 13-2,168.0,127.079307,37.494938,2859
3,강남구 개포동 567-23,92.0,127.065835,37.477888,350
4,강남구 논현동 168-0,192.0,127.026290,37.508171,36734


In [9]:
locations = parks_reports[['위도', '경도']]
fig = px.scatter_map(parks_reports, lat = '위도', lon = '경도',
                        size="총주차면", size_max=50,
                        color="민원 수",
                        zoom=11,
                     )
                     
fig.update_layout(map_style="open-street-map")

fig.show()

In [ ]:
# plot a scatter plot of parks_complaints with 총주차면 and 민원 수
fig = px.scatter(parks_reports, x='총주차면', y='민원 수',
                # size='총주차면',
                #  log_x=True, size_max=60,
                title='총주차면 vs 민원 수')

fig.update_layout(
    xaxis_title='총주차면',
    yaxis_title='민원 수',
    legend_title='1시간 요금',
    template='plotly_white'
)
fig.show()

In [22]:
# detect outliers in the scatter plot and visualize them
from scipy import stats
z_scores = np.abs(stats.zscore(parks_reports[['총주차면', '민원 수']]))
outliers = (z_scores > 3).any(axis=1)

df_outliers = parks_reports[outliers].reset_index(drop=True)
df_outliers

,주소,총주차면,경도,위도,민원 수
0,강남구 논현동 168-0,192.0,127.026290,37.508171,36734
1,강남구 청담동 77-27,3.0,127.041404,37.522200,40468
2,강남구 청담동 77-84,5.0,127.042602,37.518215,44435
3,강서구 화곡동 1030-33,27.0,126.839735,37.543908,59466
4,강서구 화곡동 1045-0,99.0,126.834751,37.541749,54456
5,강서구 화곡동 1056-2,25.0,126.840754,37.544222,51934
6,마포구 동교동 164-2,1.0,126.924737,37.556365,42914
7,마포구 동교동 165-1,1.0,126.924158,37.556754,45508
8,마포구 동교동 166-11,1.0,126.924706,37.556789,43605
9,마포구 동교동 192-6,1.0,126.923867,37.556051,45796


In [23]:
fig = px.scatter(outliers_df, x='총주차면', y='민원 수',

                title='Outliers in 총주차면 vs 민원 수')
fig.update_layout(
    xaxis_title='총주차면',
    yaxis_title='민원 수',
    template='plotly_white'
)
fig.show()


In [26]:
fig = px.scatter_map(df_outliers, lat="위도", lon="경도", hover_data=["총주차면", "민원 수"])
fig.update_traces(marker=dict(size=12, color='red', opacity=0.8))
fig.update_layout(map_style="open-street-map")
# fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

---

In [28]:
cctv_path = 'cctv.csv'
cctv = pd.read_csv(cctv_path)
cctv.head()

,고정형CCTV지번주소,latitude,longitude
0,양천구 목4동 762-10,37.535891,126.870566
1,양천구 목4동 797-8,37.532740,126.867191
2,양천구 신월2동 496,37.524535,126.848224
3,양천구 신월7동 928-1,37.522349,126.833529
4,양천구 목1동 917,37.528982,126.874597


In [31]:
fig = px.scatter_map(cctv, lat="latitude", lon="longitude", zoom=11)
fig.update_traces(marker=dict(size=12, opacity=0.8))
fig.update_layout(map_style="open-street-map")
# fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()